In [1]:
import pandas as pd

In [2]:
data_url = './make_file/광진구_자동차도로_전체.csv'

In [3]:
data = pd.read_csv(data_url)
data.head()

,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부
0,"['광장로', '광장앞지하차도']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN
1,워커힐로,주거지 도로,37.545509,127.097772,37.546539,127.098053,37.546024,127.097913,NaN,NaN,NaN,516647748,NaN,NaN,NaN,NaN
2,워커힐로,주거지 도로,37.545509,127.097772,37.543981,127.097208,37.544745,127.097490,NaN,NaN,NaN,516647752,NaN,NaN,NaN,NaN
3,자양번영로,3차로,37.533200,127.075680,37.533816,127.075863,37.533508,127.075772,NaN,NaN,NaN,37401581,30.0,4.0,NaN,NaN
4,자양번영로,3차로,37.533200,127.075680,37.532597,127.075517,37.532898,127.075599,NaN,NaN,NaN,516723466,NaN,NaN,NaN,NaN


In [4]:
# start_latitude = []
# start_longitude = []
# start_altitude =[]

# end_latitude = []
# end_longitude = []
# end_altitude = []

In [5]:
start_latitude = data['시작점_위도'].to_list()
start_longitude = data['시작점_경도'].to_list()

end_latitude = data['종료점_위도'].to_list()
end_longitude = data['종료점_경도'].to_list()

start_altitude = []
end_altitude = []

In [6]:
print(len(start_latitude))
print(len(start_longitude))
print(len(end_latitude))
print(len(end_longitude))

6288
6288
6288
6288


## 구글 API 활용

In [7]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()


사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
Name: count, dtype: int64

In [8]:
GOOGLE_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'GOOGLE_MAP'].values[0][1]  

In [9]:
import requests
import time
import pandas as pd 

# Google Elevation API 호출 함수
def google_api(latitudes, longitudes, mode):
    global start_altitude, end_altitude
    
    url = f"https://maps.googleapis.com/maps/api/elevation/json?key={GOOGLE_API_KEY}"

    locations = list(zip(latitudes, longitudes))  # (위도, 경도) 쌍 리스트
    
    for i in range(0, len(locations), 512):  # 최대 512개씩 요청
        batch_locations = locations[i:i+512]
        locations_param = '|'.join([f"{lat},{lon}" for lat, lon in batch_locations])
        full_url = f"{url}&locations={locations_param}"
        
        response = requests.get(full_url)
            
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'OK':
                elevations = [result['elevation'] for result in data['results']]   
            
                if mode == 'start':
                    start_altitude.extend(elevations)
                elif mode == 'end':
                    end_altitude.extend(elevations)
            else:
                print("Error in response:", data['status'])
        else:
            print("API 호출 실패:", response.status_code)

        time.sleep(0.1)  # API 호출 간 지연 시간

In [10]:
print(f'전) 기점 고도 길이 : {len(start_altitude)}') 

print(f'전) 종점 고도 길이 : {len(end_altitude)}') 

전) 기점 고도 길이 : 0
전) 종점 고도 길이 : 0


In [11]:
# API 호출 (start)
print(f'기점의 고도값 불러오기')
print(f'시작')
google_api(start_latitude, start_longitude, 'start') 
print(f'종료')

기점의 고도값 불러오기
시작


종료


In [12]:
# API 호출 (end)
print(f'종점의 고도값 불러오기')
print(f'시작')
google_api(end_latitude, end_longitude, 'end')
print(f'종료')

종점의 고도값 불러오기
시작


종료


In [13]:
print(f'후) 기점 고도 길이 : {len(start_altitude)}') 
print(f'후) 종점 고도 길이 : {len(end_altitude)}') 

if (len(start_altitude) == len(end_altitude)) and (len(start_altitude) > 0) :
    print(f'결과의 길이가 같습니다.')
    print(f'성공적으로 처리되었습니다.')
else :
    print(f'데이터 처리에 실패했습니다...')

후) 기점 고도 길이 : 6288
후) 종점 고도 길이 : 6288
결과의 길이가 같습니다.
성공적으로 처리되었습니다.


In [14]:
altitude_difference = []

for i in range(len(start_altitude)) :
    result = abs(start_altitude[i] - end_altitude[i]) 
    altitude_difference.append(result)

print(len(altitude_difference))

6288


In [15]:
# 결과 데이터 추가 및 저장
data['시작점_고도'] = start_altitude
data['종료점_고도'] = end_altitude
data['고도의_차이'] = altitude_difference

output_file = './make_file/자동차_도로_고도차이_포함_데이터.csv'

try :
    data.to_csv(output_file, index=False)
    print(f'{output_file} 파일이 저장되었습니다.')
except :
    print(f'{output_file} 파일 저장에 실패했습니다.')

./make_file/자동차_도로_고도차이_포함_데이터.csv 파일이 저장되었습니다.
